<a href="https://colab.research.google.com/github/scman1/verify_gti/blob/master/Custom_OD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Custom Object Detection Yolo V3
This project was created following these sources:
 - [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/)
 - [When MathMeets Coding YouTube video](https://www.youtube.com/watch?v=hTCmL3S4Obw&t=198s). The labelling of images is skipped as the datsets used were labelled beforehand for other project.
 - [jackkcoder github repo](https://github.com/jakkcoder/training_yolo_custom_object_detection_files)
 
The project is linked to a google drive to preserve all data between runs. 

# Setup
After creating the notebook, make sure that it has GPU enabled, to take advantage of the GPU. Go to Runtime, then to change runtime and select GPU from the dropdown list and save.

The following code mounts the google drive in the colab machine and clones the darknet code to a folder inside the drive.

In [ ]:
# mount a google drive
from google.colab import drive
drive.mount('/content/drive')
# clone the darknet repository
#!git clone 'https://github.com/AlexeyAB/darknet.git' '/content/drive/MyDrive/abdknet'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Make Darknet
Make sure that the runtime environment has GPU support (Runtime>Change runtime>Select GPU>Save).

Edit the makefile to and change the values for GPU, CUDNN and OPENCV to 1. So Darknet is built using for the machine with GPU acceleration, CUDA and OpenCV.

After making sure the runtime is correct and the makefile has been edited, run make to build darknet.

In [ ]:
# go to the directory that was just coded
%cd '/content/drive/MyDrive/abdknet'
# check that we are in the right directory
!ls
# edit make file making GPU, CUDNN and OPENCV = 1
# create the make file
!make # only run once

/content/drive/MyDrive/abdknet
3rdparty	DarknetConfig.cmake.in	json_mjpeg_streams.sh  scripts
backup		darknet_images.py	LICENSE		       src
build		darknet.py		Makefile	       vcpkg.json
build.ps1	darknet_video.py	net_cam_v3.sh	       video_yolov3.sh
cfg		data			net_cam_v4.sh	       video_yolov4.sh
cmake		image_yolov3.sh		obj
CMakeLists.txt	image_yolov4.sh		README.md
darknet		include			results
chmod +x *.sh


## Copy the data for training and testing
Create a 'custom_dataset' directory on the root of the google drive to store the images and annotation files. In this case the directory is named 'rbgk' (same as our dataset name).

Copy the images and annotation files into that folder, as well as the names.txt file. names.txt contains the namesof the classes used to annotate the dataset.

Get the two python script files from this [github repo](https://github.com/jakkcoder/training_yolo_custom_object_detection_files). The scripts are used to create the custom_dataset.names, test.txt and train.txt files.
they need to be edited to set the paths where the images, txt labels and names.text files are located. If required creating-train-and-test-txt-files.py also need to be edited to match the extension of image files (it looks for .jpeg).



In [ ]:
%cd '/content/drive/MyDrive/'
#!ls
#!unzip /content/drive/MyDrive/rbgk/images.zip -d /content/drive/MyDrive/rbgk/
#!python rbgk/creating-files-data-and-name.py
#!python rbgk/creating-train-and-test-txt-files.py


/content/drive/MyDrive


## Edit cfg file
In the darknet directory (abdknet), make a copy of the yolov3.cfg file on the config (cfg) directory with the name yolov3-custom.cfg and edit it as follows.
1. uncoment the training batch and subdivision files and set values according to the number of images in training set
2. Edit the max_batches  and set the value to 2000 X [num of classes]. For the current case, we have 4 classes so whe set the value to 8000
3. Edit the values of the classes on 3 yolo layers to match the number of classes in the training set. in our case 4. Also eddit the filters value on each of the convolutional layers preceding the 3 yolo layers. The number of filters should be 3 X ([num of classes] + 5), in our case 3X(4+5) = 27.

Copy this config file to the cfg subdirectory on the darknet directory, and also the custom_dataset.names file.


## Train the network
Now run the darknet detector. Training requires the .data and .cfg files as well as the pretrained weights. Add the -dont_show modifier to prevent the environment from crashing when trying to show the graphs interactively. 



In [ ]:
!chmod +x ./abdknet/darknet
#to train from the beggining iwth pretrained darknet53
#!./abdknet/darknet detector train abdknet/cfg/rbgk_data.data abdknet/cfg/yolov3-rbgk.cfg  custom_weight/darknet53.conv.74  -dont_show
#to train from scratch using yolo weights, added clear to train from the beggining
#!./abdknet/darknet detector train abdknet/cfg/rbgk_data.data abdknet/cfg/yolov3-rbgk.cfg  custom_weight/yolov3.weights  -dont_show -clear
# to train from the last saved model:
!./abdknet/darknet detector train abdknet/cfg/rbgk_data.data abdknet/cfg/yolov3-rbgk.cfg  backup/yolov3-rbgk_last.weights  -dont_show

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3-rbgk
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortc